# Train data preparation

In [121]:
# Libraries
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

In [122]:
# We get the data from link
df = pd.read_csv('https://trein.fwrite.org/AMS-Aurora-archive/2016-01/DVS_2016-01-01.csv.xz')
df.columns = ["date", "data", "code"]

# TODO
# Get the data for each day of the whole 2016
# Maybe use the link in a loop and just replace the name of the file with the date for each day? maybe?

In [146]:
# This function iterates through each row (day) in the given dataset
# The dataset, which in this case is one whole day is parsed as a parameter to the function
# Two lists, ritstations and trains are created at the beginning. Then in the loop, ritstation and train objects are created and added to these lists.
# Later on, the lists are combined and used in dataframe for visualization.
rows = []

def GetTrainDayData(smalldf):
    # The xml data
    xml_data = smalldf["data"]
    
    # It goes through every row in the data (1 day)
    for i,train_data in enumerate(xml_data):
        root = ET.fromstring(train_data)

        # Check if there is always root
        if len(root) != 0:
            
            # This is the prefix for each element
            prefix = './/{urn:ndov:cdm:trein:reisinformatie:data:2}'
            # The data wrapper
            productDVS = root.find('{}ReisInformatieProductDVS'.format(prefix))
            # Getting the version of the data for the day (just shows how to get attributes)
            data_version = productDVS.attrib['Versie']

            # the wrapper that contain the ritstation and the train
            wrapper = root.find('{}DynamischeVertrekStaat'.format(prefix))
            
            # All data needed is extracted and put in variables here
            station_code = wrapper.find('{}StationCode'.format(prefix)).text
            station_name = wrapper.find('{}KorteNaam'.format(prefix)).text
            train_number = wrapper.find('{}TreinNummer'.format(prefix)).text
            train_type = wrapper.find('{}TreinSoort'.format(prefix)).text
            train_company = wrapper.find('{}Vervoerder'.format(prefix)).text
            train_destination = wrapper.find('{}TreinEindBestemming'.format(prefix)).text
            train_planned_departure = wrapper.find('{}VertrekTijd'.format(prefix)).text
            train_delays = wrapper.find('{}ExacteVertrekVertraging'.format(prefix)).text
            
            rows.append({"Station Code":station_code,"Station Name":station_name,"Train Number":train_number,"Train Type":train_type,"Train Company":train_company,"Train Destination":train_destination,"Train Planned Departure":train_planned_departure,"Train Delay": train_delays})


In [147]:
smalldf = df.head(390)
GetTrainDayData(smalldf)



In [148]:
dffff = pd.DataFrame(rows,columns=["Station Code","Station Name","Train Number","Train Type", "Train Company", "Train Destination", "Train Planned Departure","Train Delay"])
dffff

,Station Code,Station Name,Train Number,Train Type,Train Company,Train Destination,Train Planned Departure,Train Delay
0,HLMS,Spaarnwde,14882,Sprinter,NS,None,2016-01-01T00:10:00Z,PT0S
1,NSCH,Nweschans,37598,Stoptrein,Arriva,None,2016-01-01T00:10:00Z,PT0S
2,DVD,Duivendt,7495,Sprinter,NS,None,2016-01-01T00:11:00Z,PT0S
3,DDRS,Stadspldrs,36795,Stoptrein,Arriva,None,2016-01-01T00:11:00Z,PT0S
4,EMNZ,Emmen Z,8083,Stoptrein,Arriva,None,2016-01-01T00:11:00Z,PT0S
...,...,...,...,...,...,...,...,...
239,ASD,A'dam C,1406,Intercity,NS,None,2016-01-01T01:17:00Z,PT2M3S
240,ASD,A'dam C,1406,Intercity,NS,None,2016-01-01T01:17:00Z,PT2M3S
241,GVC,Den Haag C,1405,Intercity,NS,None,2016-01-01T01:53:00Z,PT0S
242,LEDN,Leiden C,1405,Intercity,NS,None,2016-01-01T01:23:00Z,PT2M33S
